In [ ]:
# Require auto-mount worker
from google.colab import output
from pathlib import Path
import os, multiprocessing, logging, time, threading, re


class MegaWorker(BaseWorker):
    def __init__(self, name, state):
        super().__init__(name, state)
        self.files = ['path.txt']
        self.automount = True
        self.meganum = 0
        self.meganumfiles = 0
        self.megasrc = ''
        self.megacom = 0

    def working(self):
        logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] - %(message)s')
        logthread = threading.Thread(target=self.megalog)
        with open('/content/path.txt', 'r') as f:
            content = f.read().split('\n')
        if len(content) < 5:
            print('PARA corrupted')
            self.success = False
            return
        USERNAME = content[0]
        PASSWORD = content[1]
        FROM = content[2]
        TO = content[3]
        PARAS = content[4]
        self.megainit(USERNAME, PASSWORD)
        logthread.start()
        self.megatransfer(FROM, TO, PARAS)
        print()
        logging.info('Transfer completed: ' + str(self.meganum) + ' / ' + str(self.meganumfiles))
    
    def megalogin(self, USERNAME, PASSWORD):
        os.environ['usrC'] = USERNAME
        os.environ['pwdC'] = PASSWORD
        !mega-login $usrC $pwdC

    def megainit(self, USERNAME, PASSWORD):
        if not os.path.exists("/usr/bin/mega-cmd"):
            print("Installing MEGA ...")
            !apt-get -y update
            !apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https
            !curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb
            !dpkg -i /var/cache/apt/archives/MEGAcmd.deb
            output.clear()
            print("MEGA is installed.")
        print('----- Login in -----')
        p = multiprocessing.Process(target=self.megalogin, args=[USERNAME, PASSWORD, ])
        p.start()
        p.join(10)
        if p.is_alive():
            p.terminate()
            p.join()
        print('----- Completed -----')
        !mega-whoami

    def megatransfer(self, FROM, TO, PARAS):
        if not os.path.exists('/content/drive'):
            print('Worker auto-mount disabled')
            self.success = False
            return
        print('----- Checking -----')
        char = list(PARAS)
        alpha = (char[0] != '%')
        !mega-find '' | awk -F"/" '{OFS="/"; if(NF>0){NF=(NF-1);print $0}}' | sort | uniq > /content/folder.txt
        with open('/content/folder.txt', 'r') as f:
            folders = []
            for line in f:
                if line != '\n':
                    folders.append(line.strip('\n'))
            folders.append(FROM)
        !mega-find '{FROM}' > /content/file.txt
        self.megacom = 0
        with open('/content/file.txt', 'r') as f:
            files = []
            for line in f:
                raw = line.strip('\n')
                if not raw in folders:
                    f_folder, f_file = os.path.split(raw)
                    if f_folder.startswith(FROM):
                        f_folder = f_folder[len(FROM):]
                    f_dst = os.path.join(TO, f_folder.lstrip('/'))
                    if alpha:
                        if f_file[:1].lower() in char:
                            if not os.path.exists(os.path.join(f_dst, f_file)):
                                files.append(raw)
                            else:
                                self.megacom += 1
                    else:
                        if not f_file[:1].isalpha():
                            if not os.path.exists(os.path.join(f_dst, f_file)):
                                files.append(raw)
                            else:
                                self.megacom += 1
        print(self.megacom, 'completed files')
        print('----- Transfering -----')
        self.meganumfiles = len(files)
        self.meganum = 0
        if self.meganumfiles == 0:
            print('There is nothing to transfer')
        for src in files:
            self.megasrc = src
            folder_path, file_path = os.path.split(src)
            if folder_path.startswith(FROM):
                folder_path = folder_path[len(FROM):]
            dst = os.path.join(TO, folder_path.lstrip('/'))
            if not os.path.isdir(dst):
                Path(dst).mkdir(parents=True, exist_ok=True)
            self.meganum += 1
            print()
            logging.info('Transfering file: ' + str(self.meganum) + ' / ' + str(self.meganumfiles) + '\n' + self.megasrc)
            src = re.escape(src)
            !mega-get "{src}" "{dst}"

    def megalog(self):
        while True:
            time.sleep(60)
            print()
            logging.info('Transfering: ' + str(self.meganum) + ' / ' + str(self.meganumfiles) + '\nCompleted: ' + str(self.megacom) + '\n' + self.megasrc)